In [ ]:
# TODO User configurations
#geo_json_file = 'data-1.json'
#geom_column = 'LGA_MAIN11'
#data_column = 'Tot_1941_1950'
#src_geom = 'LGA_2011'
#dst_geom = 'LGA_2016'

# Please select a GeoJSON file from the JupyteLab workspace.
geo_json_file = 'data574807371901850620.json'
# Please select from the input file the geometry column.
geom_column = 'LGA_MAIN11'
# Please select from the data file the data column.
data_column ='P_Vis_within_Aus_Tot'
# Please select the 'from' geometry from a list of available conversions.
src_geom = 'LGA_2011'
# Please select the 'to' geometry from a list of available conversions.
dst_geom = 'LGA_2016'

In [ ]:
BASE_URL = 'https://ydyr.info/api/v1'

In [ ]:
AUTH = ('<YDYR username>', '<YDYR password>')

In [ ]:
# GeoJSON files directory
GEO_JSON_PATH = 'data'

In [ ]:
# Downloaded result file directory
OUTPUT_PATH = 'results'

In [ ]:
import json
import time
import requests
import os
import folium
import pandas as pd

In [ ]:
# post geo json data:
input_file = '/'.join((GEO_JSON_PATH, geo_json_file))
with open(input_file, 'r') as d:
    post_data = json.load(d)

In [ ]:
def get_side_data(geom_column):
    year = int(geom_column.split('_')[-1])
    side_data_year = 2011 if (abs(year - 2016) >= abs(year-2011)) else 2016
    return "BCP_" + str(side_data_year)

In [ ]:
# post parameters:
post_parms = {
    'geom_column': geom_column,
    'data_column': data_column,
    'src_geom': src_geom,
    'dst_geom': dst_geom,
    'side_data': get_side_data(src_geom), #TODO!!!
    'averaged_counts': False # XXX TODO
}

In [ ]:
def get_disaggration(data, parms):
    res = requests.post(
        '/'.join((BASE_URL, 'disaggregate.geojson')),
        auth=AUTH,
        params=parms,
        json=data)
    assert res.status_code == 202, 'job failed to queue'
    job_id = res.json()
    print("Your submitted job id is", job_id)
    print()
    
    while res.status_code == 202:
        time.sleep(2)
        res = requests.get(
            '/'.join((BASE_URL, 'status', job_id)),
            auth=AUTH)
    assert res.status_code == 200, 'job failed to finish'
    print("Get the job status report:", res.json()['report'])
    
    # result_id != job_id
    result_id = res.json()['key']
    res = requests.get(
        '/'.join((BASE_URL, 'download', result_id)),
        auth=AUTH,
        params={'format': 'json'})
    assert res.status_code == 200, 'failed to download result'
    #print("Get downloaded data:", res.json())
    
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
    output_file = '/'.join((OUTPUT_PATH, job_id+'.json'))
    with open(output_file, 'w') as f:
        json.dump(res.json(), f)
    print("Done! Your job result is downloaded at", output_file)
    print()

In [ ]:
def get_capability():
    res = requests.get(
        '/'.join((BASE_URL, 'capability')),
        auth=AUTH,
        params={'format': 'json'})
    assert res.status_code == 200, 'capability request failed'
    
    if res.ok:
        return(res.json())

In [ ]:
data = {'id':[], data_column:[]}
for feature in post_data.get('features'):
    data["id"].append(feature.get("id"))
    data[data_column].append(feature.get('properties').get(data_column))
                    
data_pd = pd.DataFrame.from_dict(data)
print(data_pd[:10])

In [ ]:
m = folium.Map(location=[-23.6980, 133.8807], zoom_start=4)

folium.Choropleth(
    geo_data=input_file,
    name='choropleth',
    data= data_pd,
    columns=['id', data_column],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name=data_column
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
print("Before Aggregation:")
m

In [ ]:
get_capability()

In [ ]:
get_disaggration(post_data, post_parms)